In [93]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import pandas as pd
import os

## Define parameters

In [94]:
learning_rate = 1e-3
input_size = 100
output_size = 100
batch_size = 1
num_epochs = 100
num_workers = 8
train_dir = os.path.join(os.getcwd(), 'data', 'train')
test_dir = os.path.join(os.getcwd(), 'data', 'test')
dtype_to_use = torch.float32

## Define the neural network architecture
Layer options (More info at https://pytorch.org/docs/stable/nn.html):
+ Linear: fully connected layer
+ Conv1d/Conv2d: Convolutional layers
+ BatchNorm2d/LayerNorm/InstanceNorm2d: Normalization layers
+ Dropout: Dropout layer
+ MaxPool2d/AvgPool2d: Pooling layers

In [95]:
class NN(pl.LightningModule):
    def __init__(self, learning_rate, input_size, output_size):
        super(NN, self).__init__()
        self.learning_rate = learning_rate
        self.layer1 = nn.Linear(input_size, 20)
        self.layer2 = nn.Linear(20, output_size)
        self.relu = nn.ReLU()
        self.loss_fn = nn.MSELoss()

        self.training_step_outputs = []
        self.testing_step_outputs = []
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.layer2(x)
        return x

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    
    def on_train_epoch_end(self):
        if self.current_epoch == self.trainer.max_epochs - 1:
            # Calculate distances for the last epoch
            avg_distances, max_distances = self.calculate_distances(self.training_step_outputs)
            for i, avg_distance in enumerate(avg_distances):
                self.logger.experiment.add_scalar("train/avg_distance", avg_distance, i)
                self.logger.experiment.add_scalar("train/max_distance", max_distances[i], i)	

    def on_test_epoch_end(self):
        if self.current_epoch == self.trainer.max_epochs - 1:
            # Calculate distances for the last epoch
            avg_distances, max_distances = self.calculate_distances(self.testing_step_outputs)
            for i, avg_distance in enumerate(avg_distances):
                self.logger.experiment.add_scalar("test/avg_distance", avg_distance, i)
                self.logger.experiment.add_scalar("test/max_distance", max_distances[i], i)

            

    def calculate_distances(self, output_target_pairs):
        avg_distances, max_distances = [], []
        for output, target in output_target_pairs:
            avg_distance, max_distance = self.calculate_distance(output, target)
            avg_distances.append(avg_distance)
            max_distances.append(max_distance)

        return avg_distances, max_distances

    def calculate_distance(self, output, target):
        abs_diff = torch.abs(output - target)
        avg_distance = torch.mean(abs_diff)
        max_distance = torch.max(abs_diff)
        return (avg_distance.item(), max_distance.item())
    

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)
        loss = self.loss_fn(outputs, targets)
        self.log("train_loss", loss)
        if self.current_epoch == self.trainer.max_epochs - 1:
            self.training_step_outputs.append((outputs, targets))
        else:
            self.training_step_outputs = []
        return loss
    
    def test_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)
        loss = self.loss_fn(outputs, targets)
        self.log("test_loss", loss)
        if self.current_epoch == self.trainer.max_epochs - 1:
            self.testing_step_outputs.append((outputs, targets))
        else:
            self.testing_step_outputs = []
        return loss


## Custom dataset class

In [96]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Assuming each item in the dataset is a tuple of (input, output)
        sample = self.data[index]
        input_array, output_array = sample[0], sample[1]

        # Convert to PyTorch tensors
        input_tensor = torch.Tensor(input_array)
        output_tensor = torch.Tensor(output_array)

        return input_tensor, output_tensor
    
class CustomDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, num_workers):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_ds = None
        self.test_ds = None

    def prepare_data(self):
        pass

    #Arrays are transposed to make input size 100 instead of 2
    def setup(self, stage):
        train_data = []
        test_data = []
        for subdir in os.listdir(train_dir):
            if ("188root" not in subdir):
                continue
            input_output = (pd.read_csv(os.path.join(train_dir, subdir, 'polar.csv')).values.transpose(), pd.read_csv(os.path.join(train_dir, subdir, 'coords.csv')).values.transpose())
            train_data.append(input_output)
            break
        for subdir in os.listdir(test_dir):
            input_output = (pd.read_csv(os.path.join(test_dir, subdir, 'polar.csv')).values.transpose(), pd.read_csv(os.path.join(test_dir, subdir, 'coords.csv')).values.transpose())
            test_data.append(input_output)
            break

        self.train_ds = CustomDataset(train_data)
        self.test_ds = CustomDataset(test_data)

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            #num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            #num_workers=self.num_workers,
        )

In [97]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lightning_model = NN(learning_rate=learning_rate, input_size=input_size, output_size=output_size).to(device)


dm = CustomDataModule(batch_size, num_workers)

# Train and plot loss over epochs, as well as average and maximum difference for every sample in the last epoch

In [100]:
tb_logger = TensorBoardLogger("logs", name="results")
trainer = pl.Trainer(max_epochs=num_epochs, logger=tb_logger)  # Adjust parameters as needed

# Train the model
trainer.fit(lightning_model, dm)
%tensorboard --logdir logs/results/

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | layer1  | Linear  | 2.0 K 
1 | layer2  | Linear  | 2.1 K 
2 | relu    | ReLU    | 0     
3 | loss_fn | MSELoss | 0     
------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params
0.016     Total estimated model params size (MB)
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number 

Epoch 99: 100%|██████████| 1/1 [00:16<00:00, 16.03s/it, v_num=8] 

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:31<00:00, 31.10s/it, v_num=8]


UsageError: Line magic function `%tensorboard` not found.


# Get results from testing:


In [99]:
trainer.test(lightning_model, dm)


c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x95 and 100x20)